# Pretraining - Smart Contract Training Dataset Collection

This notebook collects smart contract bytecode and decompiled opcodes for normal and malicious contract classification.

Pretraining contracts are gathered from Zettablock and malicious contracts from [Forta Network's labelled datasets github repo]("https://github.com/forta-network/labelled-datasets").

In [ ]:
import logging
import pickle
import os
import requests
import json
import time

from evmdasm import EvmBytecode
import pandas as pd
from tqdm import tqdm
from web3 import Web3
from dotenv import load_dotenv

# Load secrets
dotenv_path = '../.env'
load_dotenv(dotenv_path)

tqdm.pandas()
# disable warning logs from evmdasm tool
logging.getLogger("evmdasm").setLevel(logging.CRITICAL)

zettablock_data_file = "/data/forta/ethereum/text/pretraining/zettablock_data"
processed_data_file = "/data/forta/ethereum/text/pretraining/big_pretrain_data"

# Your Zettablock API key
API_KEY = os.environ.get("ZETTABLOCK_API_KEY")
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    # credentials
    "X-API-KEY": API_KEY
}

# Zettablock endpoint
ZETTABLOCK_DATA_LAKE_ENDPOINT = os.environ.get("ZETTABLOCK_DATA_LAKE_ENDPOINT")

# Configure the blockchains we are interested in
blockchains = ["ethereum_mainnet", "polygon_mainnet", "bsc_mainnet"]

# Final training and validation files
train_file_path = "/data/forta/ethereum/text/pretraining/pretraining_train.csv"
val_file_path = "/data/forta/ethereum/text/pretraining/pretraining_val.csv"

In [ ]:
# Code taken from Zettablock tutorials
# check response until success or failed is returned
def get_response(queryrun_id):
	import time
	i = 1
	queryrun_status_endpoint = f'https://api.zettablock.com/api/v1/queryruns/{queryrun_id}/status'
	while True:
		res = requests.get(queryrun_status_endpoint, headers=headers)
		state = json.loads(res.text)['state']
		if state == 'SUCCEEDED' or state == 'FAILED':
			return state
		time.sleep(i)
		i += 1

def download_file(url: str, local_file: str, headers=None, params=None):
    resp = requests.get(url, stream=True, headers=headers, params=params)
    total = int(resp.headers.get('content-length', 0))
    with open(local_file, 'ab') as file, tqdm(
        desc=local_file,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)

def call_zettablock_api(query_text, blockchain):
    # Get Smart Contract Data from Zettablock for several blockchains
    query = {"query": query_text, "resultCacheExpireMillis": 86400000}
    
    # Create a query with SQL statement, and get query id
    res = requests.post(data_lake_query_endpoint, headers=headers, data=json.dumps(query))
    print(res.text)
    
    # Trigger the query by query id, and get queryrun id
    query_id = res.json()['id']
    data_lake_submission_endpoints = f'https://api.zettablock.com/api/v1/queries/{query_id}/trigger'
    res = requests.post(data_lake_submission_endpoints, headers=headers, data='{}')
    
    # Check status using queryrun id
    queryrun_id = res.json()['queryrunId']
    
    if get_response(queryrun_id) == 'SUCCEEDED':
        # Fetch result from queryrun id
        params = {'includeColumnName': 'true'}
        queryrun_result_endpoint = f'https://api.zettablock.com/api/v1/stream/queryruns/{queryrun_id}/result'
        # if the result is huge, consider using stream and write to a file
        download_file(queryrun_result_endpoint, zettablock_data_file+"_"+blockchain+".csv", headers=headers, params=params)
    else:
        print('query failed, please check status message for details')
        print(res.json())

In [ ]:
# Code provided by the Forta team
# Decompile and dissasemble the smart contract bytecode
def get_opcodes(creation_bytecode) -> str:
    bytecode = creation_bytecode
    if bytecode is None:
        return ''

    try:
        opcodes = EvmBytecode(bytecode).disassemble()
    except Exception:
        return ''
    
    return " ".join([str(op).strip() for op in opcodes])

In [ ]:
# Code provided by the Forta team
# Filter opcodes to get the best features
def get_exp_2_features(row):
    creator = row['contract_creator']
    opcodes = row['decompiled_opcodes'].split()
    mask = '0xffffffffffffffffffffffffffffffffffffffff'
    features = []
    for i in range(len(opcodes)-1):
        first = opcodes[i]
        second = opcodes[i+1]
        if not first.startswith('0x'):
            token = first
            if first.startswith('UNKNOWN') or first.startswith('INVALID'):
                token = first.split('_')[0]
            features.append(token)
        elif first == 'PUSH4':
            features.append(second)
        elif first == 'PUSH20':
            if second == creator:
                features.append('creator')
            elif second == mask:
                features.append(mask)
            else:
                features.append('address')
        elif first == 'PUSH32':
            features.append(second)
    return " ".join(features)

In [ ]:
# Create queries for the supported blockchains
# They don't have all the same available data
def get_query(blockchain):
    query_text = ""
    if blockchain == "ethereum_mainnet":
        query_text = '''
            SELECT contract.address as contract_address,
                   contract.name as contract_name,
                   contract.creator as contract_creator,
                   tags.name as contract_tag_name, 
                   tags.type as contract_type,
                   contract.code as contract_code
            FROM {}.contracts contract LEFT JOIN {}.labels tags ON tags.address = contract.address
            LIMIT 30000
        '''.format(blockchain, blockchain)
    elif blockchain == "polygon_mainnet":
        query_text = '''
            SELECT contracts.address as contract_address,
                   mappings.contract_name as contract_name,
                   contracts.creator_address as contract_creator,
                   mappings.contract_category as contract_type,
                   contracts.bytecode as contract_code
            FROM polygon_mainnet.contract_creations contracts LEFT JOIN polygon_mainnet.contract_mappings mappings ON mappings.contract_address = contracts.address
            LIMIT 30000
        '''.format(blockchain, blockchain)
    else:
        query_text = '''
            SELECT contracts.address as contract_address,
                   contracts.creator_address as contract_creator,
                   contracts.bytecode as contract_code
            FROM bsc_mainnet.contract_creations contracts
            LIMIT 30000
        '''.format(blockchain, blockchain)
    return query_text

# Get contracts and filter the data
def get_pretrain_contracts():
    # Get data from Zettablock for 3 different EVM compatible blockchains
    for blockchain in blockchains:
        if not os.path.exists(zettablock_data_file+"_"+blockchain+".csv"):
            print("Dowloading data from %s..." % (blockchain))
            call_zettablock_api(get_query(blockchain), blockchain)
        if not os.path.exists(processed_data_file+"_"+blockchain+".csv"):
            """Collects contracts from Zettablock and its decompiled opcodes.""" 
            chunksize = 10 ** 6
            print("Decompiling and extracting opcodes for blockchain %s:" % blockchain )
            with pd.read_csv(zettablock_data_file+"_"+blockchain+".csv", chunksize=chunksize) as contract_reader:
                for contracts in contract_reader:
                    contracts['decompiled_opcodes'] = contracts['contract_code'].progress_apply(get_opcodes)
                    contracts = contracts[(contracts['decompiled_opcodes'].notna()) & (contracts['decompiled_opcodes'] != '')]
                    contracts.drop_duplicates('contract_address', inplace=True)
                    contracts['decompiled_opcodes'] = contracts.progress_apply(get_exp_2_features, axis=1)
                    # Store data so we don't have to download it all the time
                    contracts.to_csv(processed_data_file+"_"+blockchain+".csv", mode='a')
        else:
            print("%s already exists." % processed_data_file+"_"+blockchain+".csv")

In [ ]:
# Actually run the collection code
get_pretrain_contracts()

In [ ]:
# Finally prepare data for pretraining phase
pretraining_data = {}
for blockchain in blockchains:
    # Load data from disk
    pretraining_data[blockchain] = pd.read_csv(processed_data_file+"_"+blockchain+".csv")

# Concat data in the same pandas variable
pretraining_data = pd.concat(list(pretraining_data.values()))
# Suffle data so we have mixed and heterogeneos samples from all the blockchains
print(pretraining_data.shape)
pretraining_data = pretraining_data.sample(frac = 1)

training_samples = 60000
# Save the data to disk
training_data = pretraining_data[:training_samples]
validation_data = pretraining_data[training_samples:]
training_data['decompiled_opcodes'].to_csv(train_file_path, sep='\t', index=False)
validation_data['decompiled_opcodes'].to_csv(val_file_path, sep='\t', index=False)